<a href="https://colab.research.google.com/github/AkihiroYamanishi/MathematicalOptimization/blob/main/1_5%E5%8F%8C%E5%AF%BE%E5%95%8F%E9%A1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Groubiをインポート
!pip install gurobipy-stubs
from gurobipy import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 29.6 MB/s eta 0:00:00


**問題の設定**

In [2]:
#d：需要量
d = {1:80, 2:270, 3:250, 4:160, 5:180}
#M：容量
M = {1:500, 2:500, 3:500}
#I：顧客の番号のリスト
I = [1, 2, 3, 4, 5]
#J：工場の番号リスト
J = [1, 2, 3]

上記の別の書き方（より記述が少なく済む）→Groubiの記載方法

In [3]:
I,d = multidict({1:80, 2:270, 3:250, 4:160, 5:180})
J, M = multidict({1:500, 2:500, 3:500})

In [4]:
c = {(1,1):4, (1,2):6, (1,3):9, (2,1):5, (2,2):4, (2,3):7, (3,1):6, (3,2):3, (3,3):4, (4,1):8, (4,2):5, (4,3):3, (5,1):10, (5,2):8, (5,3):4,}  

**モデルの作成**

In [5]:
#モデルに名前を付ける　今回はlo1
model = Model("transportation")

Restricted license - for non-production use only - expires 2024-10-28


**変数の定義**

In [6]:
#空の辞書xを作成
x = {}
#空の辞書xに変数のオブジェクトを保管していく
for i in I:
  for j in J:
    x[i, j] = model.addVar(vtype = "C", name = "x(%s,%s)" %(i, j))

**モデルの更新**

In [7]:
#Groubiにモデルが変更されたことを伝えるメソッド（制約を追加する前には必ず行わなければならない）→モデルが変更されるたびにGroubi内のデータ構造を更新していると時間を要するために導入されたGroubiの重要な仕様
model.update()

**制約条件の追加**

In [8]:
#制約条件をモデルに追加
for i in I:
  model.addConstr(quicksum(x[i, j] for j in J)==d[i], name="Demand(%s)" % i)

In [9]:
for j in J:
  model.addConstr(quicksum(x[i, j] for i in I)<=M[j], name="Capacity(%s)" % j)  

**目的関数の追加**

In [10]:
#目的関数の追加(最小化の場合はGRB.MINIMIZE)
model.setObjective(quicksum(c[i,j]*x[i,j] for (i,j) in x), GRB.MINIMIZE)

**最適化の実行**

In [11]:
model.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 8 rows, 15 columns and 30 nonzeros
Model fingerprint: 0xa225ad1d
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+01, 5e+02]
Presolve time: 0.02s
Presolved: 8 rows, 15 columns, 30 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.3500000e+03   2.000000e+01   0.000000e+00      0s
       1    3.3700000e+03   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.04 seconds (0.00 work units)
Optimal objective  3.370000000e+03


**目的関数値と最適解の出力**

In [12]:
#最適化の実行 最適化の際には自動的にmodelのupdateを行う
print("Optimal value:", model.ObjVal)
#
EPS = 1.e-6
for (i,j) in x:
  if x[i,j].X > EPS:
    print("sending quality %10s from factory %3s to customer %3s" %(x[i,j].X, j, i))

Optimal value: 3370.0
sending quality       80.0 from factory   1 to customer   1
sending quality       20.0 from factory   1 to customer   2
sending quality      250.0 from factory   2 to customer   2
sending quality      250.0 from factory   2 to customer   3
sending quality      160.0 from factory   3 to customer   4
sending quality      180.0 from factory   3 to customer   5


以上、1.4のコードと全く同じ
以下、双対問題固有のコード

In [13]:
print("Const.Name: Slack, Dual")
for c in model.getConstrs():
  print("%s: %s , %s" %(c.ConstrName, c.Slack, c.Pi))

Const.NAme: Slack, Dual
Demand(1): 0.0 , 4.0
Demand(2): 0.0 , 5.0
Demand(3): 0.0 , 4.0
Demand(4): 0.0 , 3.0
Demand(5): 0.0 , 4.0
Capacity(1): 400.0 , 0.0
Capacity(2): 0.0 , -1.0
Capacity(3): 160.0 , 0.0
